In [1]:
%load_ext autoreload
%autoreload 2
from panda_kinematics import PandaKinematics

import torch
import numpy as np
np.set_printoptions(2, suppress=True)
torch.set_printoptions(2, sci_mode=False)

import sys
sys.path.append('../')
from ttgo import TTGO
from manipulator_utils import test_robotics_task
from panda_cost_utils import SDF_Cost, PandaCost
from utils import test_ttgo
device = "cpu"#torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load system and environment parameters

In [2]:
file_name = 'panda_ik.pickle'

In [3]:
# Load the parameters from the training
model = torch.load(file_name,map_location=torch.device("cpu"))
tt_model = model['tt_model']
w_goal,w_obst,w_orient= model['w']
b_goal,b_obst,b_orient= model['b']
margin= model['margin']
domain = model['domains']
Rd_0= model['Rd_0']
v_d= model['v_d']
test_task = model['test_task']

key_points_weight = model['key_points_weight']
key_points_margin = model['key_points_margin']
panda = model['panda'];panda.set_device('cpu')
pandaCost = model['pandaCost']; pandaCost.set_device('cpu')
sdf_cost = model['sdf_cost']; sdf_cost.set_device('cpu')


In [6]:
key_points_margin[-1][-1]

tensor([0.02])

Define the cost/pdf function

In [7]:
def cost(x): # For inverse kinematics
    return pandaCost.cost_ik(x)[:,0]

def cost_all(x): # For inverse kinematics
    return pandaCost.cost_ik(x)


def pdf(x):
    cost_ = cost(x)
    pdf_ = torch.exp(-cost_**2)
    return pdf_

Get the TT-Model

In [9]:
# Fit the TT-model
ttgo = TTGO(domain=domain, pdf=pdf,cost=cost,device=device)
ttgo.tt_model = tt_model
ttgo.round(1e-3)
ttgo.to('cpu')

# Prepare for the task
sites_task = list(range(3))
ttgo.set_sites(sites_task)

In [21]:
torch.set_printoptions(sci_mode=True)
ttgo.tt_model.numel()

tensor(1.06e+18, dtype=torch.float64)

In [23]:
tt_model = ttgo.tt_model
n_ = 0
for core in tt_model.cores:
    n_+=core.shape[0]*core.shape[1]*core.shape[2]
n_ = torch.tensor([n_]).float()
n_

tensor([ 1.43e+07])

Test the model

In [7]:
# # Generate test set
# ns = 100
# test_task = torch.zeros(ns,3)
# for i in range(3):
#     unif = torch.distributions.uniform.Uniform(low = domain[i][0],high=domain[i][-1])
#     test_task[:,i]= torch.tensor([unif.sample() for i in range(ns)])

In [8]:
# sample_set = [1, 10, 100, 1000]
# alphas = [0.9,0.75,0.5,0]
# cut_total=0.25
# test_robotics_task(ttgo.clone(), cost_all, test_task, alphas, sample_set, cut_total,device)

.

## Visualization

In [9]:
# p.disconnect()
# Setup the robot and the environment

data_sdf = np.load('./data/sdf.npy', allow_pickle=True)[()]
sdf_matr = data_sdf['sdf_matr']  #SDF tensor
bounds = torch.tensor(data_sdf['bounds']) # Bound of the environment
env_bound = data_sdf['env_bound']  
shelf_bound = data_sdf['shelf_bound'] 
box_bound = data_sdf['box_bound'] 

data_keys = np.load('./data/sphere_setting.npy', allow_pickle=True)[()]# key_points
status_array = data_keys['status_array']
body_radius = data_keys['body_radius']
relative_pos = data_keys['relative_pos']


In [10]:
import pybullet_data
from panda_visualization_utils import *
import pybullet as p
from functools import partial
# import the environment (SDF and for graphics visualization in pybullet)
import sys
sys.path.append('../../lib')
DATA_PATH = './data'
body_sphere_path = './data/sphere_setting.npy'
sdf_path = './data/sdf.npy'
urdf_path = './data/urdf/frankaemika_new/panda_arm.urdf'

p.connect(p.GUI)
data = np.load(sdf_path, allow_pickle=True)[()]
sdf_matr = data['sdf_matr']  #SDF tensor
obstacles = data['obstacles'] #obstacles parameters
colors = [[0.8, 0.5, 0.5, 1]]*len(obstacles)
obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet (np.zeros(3), np.zeros(3), obstacles, colors=colors)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
robot_urdf ='./data/urdf/frankaemika_new/panda_arm.urdf'
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
mean_pose = 0.5*(joint_limits[0]+joint_limits[1])
set_q_std = partial(set_q,robot_id, pb_joint_indices)
rmodel = pin.buildModelFromUrdf(robot_urdf)
rdata = rmodel.createData()
pin_frame_names = [f.name for f in rmodel.frames]
ee_frame_id = rmodel.getFrameId('panda_hand_joint')
alpha = np.deg2rad(52)
quat = p.getQuaternionFromAxisAngle((0,0,1),alpha)
p.resetBasePositionAndOrientation(robot_id, (0,0,0.05), quat)

pybullet build time: Sep 20 2021 20:34:14


In [28]:
s = np.random.randint(0,test_task.shape[0]-1)
sample_xe = torch.tensor([0.5, 0.4, 0.45])#test_task[s] # torch.tensor([0.58, 0.68, 0.78])
#torch.tensor([0.70, 0.40, 0.15])#
print(sample_xe)
# torch.tensor([0.4435, 0.4347, 0.2139]) # bottom center
# torch.tensor([0.4991, 0.6301, 0.3695]) # bottom left
# torch.tensor([0.6352, 0.4708, 0.7535]) # top left
# torch.tensor([0.6789, 0.1950, 0.6976]) # top shelf center
# torch.tensor([0.6635, 0.6250, 0.2031]) # bottom, in, inaccessible
# tirch.tensor([0.4228, 0.4441, 0.8760]) # top 
# sample_xe = torch.tensor([0.8, -0.1, 0.3]) #torch.tensor([0.8, -0.3, 0.35])#torch.torch.tensor([ 0.6404, 0.2350,  0.549])#torch.tensor([ 0.7130, -0.3007, -0.1276]) #
# sample_xe[0] = 0.5
n_solutions= 1000
n_samples_tt = 10*n_solutions
n_samples_rand= 1*n_samples_tt

alpha=0.; norm=1 ; sample_replace =False; 

t1 = time.time()
samples_tt, samples_idx = ttgo.sample(n_samples=n_samples_tt, x_task=sample_xe.reshape(1,-1),alpha=alpha, norm=norm, sample_replace=sample_replace)
state_k_tt = ttgo.choose_top_k_sample(samples_tt,n_solutions)

#Optimize
state_k_tt_opt = 1*state_k_tt
for i, state in enumerate(state_k_tt):
    state_k_tt_opt[i,:],results= ttgo.optimize(state,bound=True)
t2 = time.time()

# samples_rand, _ = ttgo.sample_random(n_samples=n_samples_rand,  x_task=sample_xe.reshape(1,-1))
# state_k_rand = ttgo.choose_top_k_sample(samples_rand,n_solutions)

# #Optimize
# state_k_rand_opt = state_k_rand*1
# for i, state in enumerate(state_k_rand):
#     state_k_rand_opt[i,:],results= ttgo.optimize(state,bound=True)
t3=time.time()

print("Time taken:", (t2-t1), t3-t2)
             
c_tt =  cost_all(state_k_tt_opt)
# c_rand =   cost_all(state_k_rand_opt)

print("Cost-mean-tt:",torch.mean(c_tt,dim=0))
# print("Cost-mean-rand:",torch.mean(c_rand,dim=0))



tensor([0.50, 0.40, 0.45])


KeyboardInterrupt: 

In [ ]:
x_target = sample_xe[:3].numpy()
joint_angles_k = state_k_tt[:,3:].numpy() 
joint_angles_k_opt = state_k_tt_opt[:,3:].numpy() 

In [ ]:
# _,_,test_sphere = create_primitives(p.GEOM_SPHERE, radius = 0.02)
# p.resetBasePositionAndOrientation(test_sphere, (0,0,1.), (0,0,0,1))
_ , _,sphere_id = create_primitives(p.GEOM_SPHERE, radius = 0.02)
pos = x_target[:]

p.resetBasePositionAndOrientation(sphere_id, pos, (0,0,0,1))


k = joint_angles_k.shape[0]
dt = 1
dT = 2
for i in range(2*k):
    set_q_std(joint_angles_k[i%k])
    time.sleep(dt)
#     set_q_std(joint_angles_k_opt[i%k])
#     time.sleep(2*dt)
    

In [27]:
# torch.save(ttgo.tt_model.cores, 'panda_ttmodel.pickle')